#Gráficos interactivos para comprender el Mercado Bursátil
En este proyecto se generan distintas gráficas para poder comprender como se mueve el mercado bursátil a través del tiempo con gráficos interactivos que nos entregan la información con su fecha exacta los valores correspondientes su serie de tiempo.

Este proyecto está enfocado como programación con fines educativos.

# Primer Paso

Importar los paquetes requeridos para trabajar;

**Yfinance** Para acceder a la informacion de las acciones que deseamos en un dataframe.

**Pandas** Para trabajar y modificar los dataframe.

**Plotly** Para generar gráficos interactivos.

In [1]:
import yfinance as yf
import pandas as pd
import plotly.express as px
import numpy as np
import plotly.graph_objects as go

# Segundo Paso
Vamos a cargar la información y limpiar a nuestro requerimiento la fecha para poder graficar la información como corresponde, de tener la fecha como se entrega originalmente no podremos graficar con claridad.

En nuestro caso vamos a cargar la información de Microsoft en las fechas del 2000-10-01 hasta el 2024-10-24 en formato `Año Mes Día`.

In [2]:
data = yf.download('MSFT', start= '2000-10-01', end='2024-10-24')

data.reset_index(inplace=True)
data['Date'] = data['Date'].dt.strftime('%Y/%m/%d')

[*********************100%***********************]  1 of 1 completed


Observamos que carga de la forma correcta al cargar `data`

In [3]:
data

Price,Date,Adj Close,Close,High,Low,Open,Volume
Ticker,,MSFT,MSFT,MSFT,MSFT,MSFT,MSFT
0,2000/10/02,18.228062,29.562500,30.406250,29.125000,30.250000,58562400
1,2000/10/03,17.438046,28.281250,29.906250,28.250000,29.781250,85374000
2,2000/10/04,17.091209,27.718750,28.281250,27.250000,28.187500,136453400
3,2000/10/05,17.071945,27.687500,28.625000,27.625000,27.750000,81099400
4,2000/10/06,17.129753,27.781250,28.375000,27.375000,27.906250,61794000
...,...,...,...,...,...,...,...
6048,2024/10/17,416.720001,416.720001,422.500000,415.589996,422.359985,14820000
6049,2024/10/18,418.160004,418.160004,419.649994,416.260010,417.140015,17145300
6050,2024/10/21,418.779999,418.779999,418.959991,413.750000,416.119995,14206100


# Tercer Paso
Vamos a cargar algo simple en este caso queremos el precio de cierre ajustado de la acción con un gráfico de línea a través del tiempo para comprender la tendencia de la acción.

In [4]:
# Extraer las columnas 'Fecha' y 'Cerrar Ajuste'
dates = data[('Date', '')]
adj_close = data[('Adj Close', 'MSFT')]

# Crea una figura de Plotly
fig = go.Figure(data=[go.Scatter(x=dates, y=adj_close)])

# Set the layout (optional)
fig.update_layout(title='MSFT Precio De Cierre Ajustado',
                  xaxis_title='Fecha',
                  yaxis_title='Precio ($USD)')

# Display the figure
fig.show()

Comprendemos que el valor de las acciones de Microsoft tiene saltos bastantes grandes a medida que pasa el tiempo para el alza, podemos decir que la tecnología a medida del 2015 surge un boom constante.

Ahora que aprendimos como graficar vamos a realizar algo mucho mejor con otra acción del área de tecnología conocida.

In [5]:
apple = yf.Ticker("AAPL")
df = apple.history(period="max")
df.reset_index(inplace=True)
df['Date'] = df['Date'].dt.strftime('%Y/%m/%d')
df['20wma'] = df['Close'].rolling(window = 140).mean()
df['50wma'] = df['Close'].rolling(window = 350).mean()
df['200wma'] = df['Close'].rolling(window = 1400).mean()

Vamos a generar un gráfico de velas que es muy utilizado para la toma de decision puesto nos entrega una informacion robusta de como se mueve sus valores.

Al igual que vamos a ver sus medias móviles, en semanas, para ver tendencias a corto plazo, mediano y largo.

De forma estética se genera un fondo negro para la forma clásica de ver esta información sea semejante.

In [6]:
fig = go.Figure(data=[go.Candlestick(x=df['Date'],
                open=df['Open'], high=df['High'],
                low=df['Low'], close=df['Close'],
                name = 'AAPL')])

fig.add_trace(
    go.Scatter(
        x=df['Date'],
        y=df['20wma'],
        line = dict(color = '#e0e0e0'),
        name = '20-Semanas MA'
    )
)

fig.add_trace(
    go.Scatter(
        x=df['Date'],
        y=df['50wma'],
        line = dict(color = '#0011ff'),
        name = '50-Semanas MA'
    )
)

fig.add_trace(
    go.Scatter(
        x=df['Date'],
        y=df['200wma'],
        line = dict(color = '#f2ff00'),
        name = '200-Semanas MA'
    )
)

fig.update_layout(template = 'plotly_dark')
fig.update_layout(xaxis_rangeslider_visible=False,
                  title='Gráfico de velas Apple',
                  xaxis_title='Fecha',
                  yaxis_title='Precio ($USD)')
fig.show()

Otra información importante para alguien interesado en el mercado de acciones es como se mueve el número de acciones negociadas con el tiempo, para esto utilizamos la variable `Volume` que nos entrega exactamente lo deseado

In [7]:
fig = px.bar(df, x="Date", y="Volume", title="Número de acciones negociadas Apple")
fig.update_layout(template = 'plotly_dark',
                  yaxis_title='Volumen')
fig.show()

Notamos que la fecha con mayor volumen corresponde a el `2000/09/29` con 7.42 billones es el dia con mas volumen en la historia de Apple.

#Conclusión

¿Para que utilizar este tipo de gráficos interactivos? La razón es para comprender mejor como se mueve a través del tiempo aparte de poder seleccionar la fecha que nuestros nos plazca, otra razón es la posible integración a un sitio web para un dashboard con la información.